In [32]:
import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [33]:
import os
import sys
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from pandas import CategoricalIndex as cat

#--
os.chdir(sys.path[0])
#os.chdir("/Users/Akash")
#os.chdir("/../../../../../Volumes/peacd/NEEL")

#--
original_data = pd.read_csv("data.csv")

#--
missing_value_df = pd.DataFrame({'percent_missing': original_data.isnull().sum() * 100 / len(original_data)})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)

#--
cols_with_missing_value = missing_value_df[missing_value_df['percent_missing']>72].index.tolist()
filtered_data = original_data.drop(columns=cols_with_missing_value)

#--
variables_list = pd.read_excel("Variables_List.xlsx")

#--
x_var = []
y_var = []
for i in range(filtered_data.shape[1]):
    index = variables_list.Variable[variables_list.Variable == filtered_data.columns.values[i]].index.astype(int)[0]
    if (variables_list['Type'].iloc[index] == "exp"):
        x_var.append(variables_list.Variable[index])
    elif (variables_list['Type'].iloc[index] == "resp"):
        y_var.append(variables_list.Variable[index])
        
#--
x_filtered_df = filtered_data[x_var]
y_filtered_df = filtered_data[y_var]

#--
catX_var = []
numX_var = []
for i in range(x_filtered_df.shape[1]):
    index = variables_list.Variable[variables_list.Variable == x_filtered_df.columns.values[i]].index.astype(int)[0]
    if (variables_list['Class'].iloc[index] == "nom" or variables_list['Class'].iloc[index] == "ord"):
        catX_var.append(variables_list.Variable[index])
    elif (variables_list['Class'].iloc[index] == "num"):
        numX_var.append(variables_list.Variable[index])

#--
x_filtered_only_cat = x_filtered_df[catX_var]
x_filtered_only_num = x_filtered_df[numX_var]

#--
for col in x_filtered_only_cat.columns.values:
    x_filtered_only_cat[col] = x_filtered_only_cat[col].astype('category')

col = y_filtered_df.columns.tolist()
col.remove("g6SUSMAR")
for col in col:
    y_filtered_df[col] = y_filtered_df[col].astype('category')
    
#--
x_filtered_only_cat_wide = pd.get_dummies(x_filtered_only_cat)
for col in x_filtered_only_cat.columns.values:
    x_filtered_only_cat_wide.loc[x_filtered_only_cat[col].isnull(), x_filtered_only_cat_wide.columns.str.startswith(col+"_")] = np.nan

#--
x_filtered_only_cat_wide_minus_col = x_filtered_only_cat_wide.drop(['x6RELAT_3.0','x6RELAT_4.0','x6RELAT_7.0'], 1)
x_filtered_only_num_minus_col = x_filtered_only_num.drop('SUUage', 1)

#--
x_filtered_num_and_cat_minus_col = pd.concat([x_filtered_only_num_minus_col, x_filtered_only_cat_wide_minus_col], 1)

#--
from fancyimpute import KNN 
x_filled_knn = pd.DataFrame(KNN(k=3).fit_transform(x_filtered_num_and_cat_minus_col))
x_filled_knn.columns = x_filtered_num_and_cat_minus_col.columns.tolist()
x_filled_knn.head()

#--
x_filled_knn.iloc[:,36:350] = round(x_filled_knn.iloc[:,36:350], 0)

#--
X_final = x_filled_knn.copy()
for col in x_filtered_only_num_minus_col.columns.tolist():
    colm = x_filled_knn[col].mean()
    colstd = x_filled_knn[col].std()
    X_final[col] = (x_filled_knn[col] - colm)/colstd




Imputing row 1/364 with 171 missing, elapsed time: 0.259
Imputing row 101/364 with 5 missing, elapsed time: 0.383
Imputing row 201/364 with 68 missing, elapsed time: 0.493
Imputing row 301/364 with 8 missing, elapsed time: 0.597


## Logistic Regression

In [34]:
def LogReg123(estimator, X, y, target):
    training_score = []
    crossVal_score = []
    testing_score = []
    total_num_var = []
    max_num_var = []
    selected_var = []
    
    df = pd.concat([X, y], 1)
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    if target == 'g6SUUMUSE3':
        df = df[df[target]!=5]
        df[target].cat.remove_categories([5], inplace=True)
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), 
                                                        df[target], 
                                                        stratify=df[target], 
                                                        test_size=0.15, random_state=0)
    
    k = len(df[target].unique().tolist())
    clf = estimator.fit(X_train, y_train)
    
    var_per_group = []
    num_per_group = []

    for j in range(k):
        var_per_group.append(X_train.columns.values[list(np.where(clf.coef_[j,:]!=0))])
        num_per_group.append(len(X_train.columns.values[list(np.where(clf.coef_[j,:]!=0))]))
    
    varMax = list(set(np.concatenate(var_per_group).ravel().tolist()))
    max_num_var.append(max(num_per_group))
    total_num_var.append(len(varMax))
    selected_var.append(np.asarray(varMax))
    
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    training_score.append(accuracy_score(y_train, y_pred_train))
    testing_score.append(accuracy_score(y_test, y_pred_test))
    crossVal_score.append(np.mean(clf.scores_[0]))
    
    X_new_train = X_train.loc[:, X_train.columns.isin(varMax)]
    X_new_test = X_test.loc[:, X_test.columns.isin(varMax)]
    clf = estimator.fit(X_new_train, y_train)
    
    var_per_group = []
    num_per_group = []

    for j in range(k):
        var_per_group.append(X_new_train.columns.values[list(np.where(clf.coef_[j,:]!=0))])
        num_per_group.append(len(X_new_train.columns.values[list(np.where(clf.coef_[j,:]!=0))]))
    
    varMax = list(set(np.concatenate(var_per_group).ravel().tolist()))
    max_num_var.append(max(num_per_group))
    total_num_var.append(len(varMax))
    selected_var.append(np.asarray(varMax))
    
    y_pred_train = clf.predict(X_new_train)
    y_pred_test = clf.predict(X_new_test)
    training_score.append(accuracy_score(y_train, y_pred_train))
    testing_score.append(accuracy_score(y_test, y_pred_test))
    crossVal_score.append(np.mean(clf.scores_[0]))
    
    diff = crossVal_score[-1] - crossVal_score[-2]
    
    while diff>0:
        X_new_train = X_train.loc[:, X_train.columns.isin(varMax)]
        X_new_test = X_test.loc[:, X_test.columns.isin(varMax)]
        clf = estimator.fit(X_new_train, y_train)
    
        var_per_group = []
        num_per_group = []

        for j in range(k):
            var_per_group.append(X_new_train.columns.values[list(np.where(clf.coef_[j,:]!=0))])
            num_per_group.append(len(X_new_train.columns.values[list(np.where(clf.coef_[j,:]!=0))]))
    
        varMax = list(set(np.concatenate(var_per_group).ravel().tolist()))
        max_num_var.append(max(num_per_group))
        total_num_var.append(len(varMax))
        selected_var.append(np.asarray(varMax))
    
        y_pred_train = clf.predict(X_new_train)
        y_pred_test = clf.predict(X_new_test)
        training_score.append(accuracy_score(y_train, y_pred_train))
        testing_score.append(accuracy_score(y_test, y_pred_test))
        crossVal_score.append(np.mean(clf.scores_[0]))
    
        diff = crossVal_score[-1] - crossVal_score[-2]
        
    
    if crossVal_score[-1] == max(crossVal_score):
        which_model = len(crossVal_score) - 1
    else:
        which_model = crossVal_score.index(max(crossVal_score)) 
    
    print("Model #",str(which_model+1),"is selected",
          "\n\nThe total number of variables is:\t",
          len(selected_var[which_model]),"\n\nThese are the variables:\n\n",
          selected_var[which_model],"\n\nMost number of variables needed to predict one of the classes in each iteration:\n",
          max_num_var,"\n\nTraining accuracy in each iteration:\n",
          training_score,"\n\nCross-validation accuracy in each iteration:\n",
          crossVal_score,"\n\nTesting accuracy in each iteration:\n",
          testing_score)
    
    global final_var
    final_var = selected_var[which_model]
    
    global train_acc
    train_acc = training_score[which_model]
    
    global test_acc
    test_acc = testing_score[which_model]
    
    global val_acc
    val_acc = crossVal_score[which_model]
    

In [35]:
def LogRegMJ(estimator, X, y, target):
    training_score = []
    crossVal_score = []
    testing_score = []
    total_num_var = []
    max_num_var = []
    selected_var = []
    
    df = pd.concat([X, y], 1)
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), 
                                                        df[target], 
                                                        stratify=df[target], 
                                                        test_size=0.15, random_state=0)
    
    k = len(df[target].unique().tolist())
    clf = estimator.fit(X_train, y_train)
    
    var_per_group = []
    num_per_group = []

    var_per_group.append(X_train.columns.values[list(np.where(clf.coef_[0,:]!=0))])
    num_per_group.append(len(X_train.columns.values[list(np.where(clf.coef_[0,:]!=0))]))
    
    varMax = list(set(np.concatenate(var_per_group).ravel().tolist()))
    max_num_var.append(max(num_per_group))
    total_num_var.append(len(varMax))
    selected_var.append(np.asarray(varMax))
    
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    training_score.append(accuracy_score(y_train, y_pred_train))
    testing_score.append(accuracy_score(y_test, y_pred_test))
    crossVal_score.append(np.mean(list(clf.scores_.values())))
    
    X_new_train = X_train.loc[:, X_train.columns.isin(varMax)]
    X_new_test = X_test.loc[:, X_test.columns.isin(varMax)]
    clf = estimator.fit(X_new_train, y_train)
    
    var_per_group = []
    num_per_group = []

    var_per_group.append(X_new_train.columns.values[list(np.where(clf.coef_[0,:]!=0))])
    num_per_group.append(len(X_new_train.columns.values[list(np.where(clf.coef_[0,:]!=0))]))
    
    varMax = list(set(np.concatenate(var_per_group).ravel().tolist()))
    max_num_var.append(max(num_per_group))
    total_num_var.append(len(varMax))
    selected_var.append(np.asarray(varMax))
    
    y_pred_train = clf.predict(X_new_train)
    y_pred_test = clf.predict(X_new_test)
    training_score.append(accuracy_score(y_train, y_pred_train))
    testing_score.append(accuracy_score(y_test, y_pred_test))
    crossVal_score.append(np.mean(list(clf.scores_.values())))
    
    diff = crossVal_score[-1] - crossVal_score[-2]
    
    while diff>0:
        X_new_train = X_train.loc[:, X_train.columns.isin(varMax)]
        X_new_test = X_test.loc[:, X_test.columns.isin(varMax)]
        clf = estimator.fit(X_new_train, y_train)
    
        var_per_group = []
        num_per_group = []

        var_per_group.append(X_new_train.columns.values[list(np.where(clf.coef_[0,:]!=0))])
        num_per_group.append(len(X_new_train.columns.values[list(np.where(clf.coef_[0,:]!=0))]))
    
        varMax = list(set(np.concatenate(var_per_group).ravel().tolist()))
        max_num_var.append(max(num_per_group))
        total_num_var.append(len(varMax))
        selected_var.append(np.asarray(varMax))
    
        y_pred_train = clf.predict(X_new_train)
        y_pred_test = clf.predict(X_new_test)
        training_score.append(accuracy_score(y_train, y_pred_train))
        testing_score.append(accuracy_score(y_test, y_pred_test))
        crossVal_score.append(np.mean(list(clf.scores_.values())))
    
        diff = crossVal_score[-1] - crossVal_score[-2]

    
    if crossVal_score[-1] == max(crossVal_score):
        which_model = len(crossVal_score) - 1
    else:
        which_model = crossVal_score.index(max(crossVal_score)) 
    
    print("Model #",str(which_model+1),"is selected",
          "\n\nThe total number of variables is:\t",
          len(selected_var[which_model]),"\n\nThese are the variables:\n\n",
          selected_var[which_model],"\n\nMost number of variables needed to predict one of the classes in each iteration:\n",
          max_num_var,"\n\nTraining accuracy in each iteration:\n",
          training_score,"\n\nCross-validation accuracy in each iteration:\n",
          crossVal_score,"\n\nTesting accuracy in each iteration:\n",
          testing_score)
    
    global final_var
    final_var = selected_var[which_model]
    
    global train_acc
    train_acc = training_score[which_model]
    
    global test_acc
    test_acc = testing_score[which_model]
    
    global val_acc
    val_acc = crossVal_score[which_model]
    

### Max use

In [36]:
estimator = LogisticRegressionCV(solver='saga',penalty='l1', Cs=[1], max_iter = 10000, scoring='accuracy',
                   multi_class='multinomial',random_state=0, cv=3, refit=True)
X = X_final
y = y_filtered_df['g6SUUMUSE1']
target = 'g6SUUMUSE1'

LogReg123(estimator, X, y, target)

Model # 2 is selected 

The total number of variables is:	 124 

These are the variables:

 ['x6mcanp2' 'x6RELAT_1.0' 'EDUC_8.0' 'x6RELAT_2.0' 'MomDiscuss'
 'x6w6MUSE1_6.0' 'c6w6use_0.0' 'g6cage' 'DadNegExp' 'c6mcanp2'
 'g6fPSTR13_5.0' 'c6use_0.0' 'x6w6MUSE1_0.0' 'g6mPSTR15_2.0' 'g2agepar'
 'c6w4muse2_2.0' 'x6w6MUSE1_2.0' 'g6fparmon' 'c6w4muse1_5.0'
 'g6fPSTR12_1.0' 'g6mPSTR10_1.0' 'g6GEN_1.0' 'g6ETHNIC_2.0' 'x6mcanp05'
 'g6fPSTR9_5.0' 'g6mpstrmar' 'EDUC_m_4.0' 'ParHist_1.0' 'x6w6MUSE1_7.0'
 'g6ETHNIC_1.0' 'c6w5muse1_7.0' 'c6w5canre_6.0' 'EDUC_m_5.0'
 'c6w6MUSE2_0.0' 'x6w6use_1.0' 'x6dep_0.0' 'EDUC_7.0' 'g6mPSTR10_3.0'
 'g6fPSTR13_1.0' 'EDUC_9.0' 'g6ETHNIC_6.0' 'x6w6can4d' 'g6mcanp1'
 'c6mcanp01' 'g6fPSTR11_1.0' 'g6mPSTR16_1.0' 'g6fPSTR9_1.0' 'EDUC_m_3.0'
 'g6mparcon' 'g6SUUcage' 'x6w6can4a' 'EDUC_d_7.0' 'g6fPSTR12_4.0'
 'c6w6MUSE1_0.0' 'g6fPSTR11_2.0' 'g6fPSTR11_4.0' 'x6mcanp01' 'g6mparmon'
 'c6mcanp1' 'c6w6MUSE2_4.0' 'c6mcanp05' 'g6mPSTR14_1.0' 'EDUC_11.0'
 'g6fparsup' 'g6mPSTR14_5.0

In [37]:
max_use_predictors_regression = final_var
max_use_scores_regression = [train_acc, val_acc, test_acc]

%store max_use_predictors_regression
%store max_use_scores_regression

Stored 'max_use_predictors_regression' (ndarray)
Stored 'max_use_scores_regression' (list)


### Past year use

In [38]:
estimator = LogisticRegressionCV(solver='saga',penalty='l1', Cs=[1], max_iter = 10000, scoring='accuracy',
                   multi_class='multinomial',random_state=0, cv=3, refit=True)
X = X_final
y = y_filtered_df['g6SUUMUSE2']
target = 'g6SUUMUSE2'

LogReg123(estimator, X, y, target)

Model # 1 is selected 

The total number of variables is:	 103 

These are the variables:

 ['x6mcanp2' 'x6RELAT_1.0' 'c6w4muse1_6.0' 'MomDiscuss' 'c6w6use_0.0'
 'c6w4use_0.0' 'g6fpstrmar' 'g6cage' 'EDUC_d_3.0' 'DadNegExp' 'c6mcanp2'
 'g6fVALMAR_7.0' 'g6fPSTR13_5.0' 'x6w6MUSE2_0.0' 'EDUC_3.0'
 'g6fPSTR10_3.0' 'g6mPSTR14_2.0' 'x6w6MUSE1_2.0' 'g6fparmon'
 'g6fPSTR9_2.0' 'c6w6candep_1.0' 'g6fPSTR12_1.0' 'g6mPSTR10_1.0'
 'g6GEN_1.0' 'g6ETHNIC_2.0' 'x6mcanp05' 'g6fPSTR9_5.0' 'g6mpstrmar'
 'ParHist_1.0' 'EDUC_m_4.0' 'c6w5canre_6.0' 'c6w6MUSE2_0.0' 'x6dep_0.0'
 'EDUC_7.0' 'c6w5muse1_1.0' 'x6w6can4d' 'c6mcanp01' 'g6mcanp1'
 'g6mPSTR16_1.0' 'g6mparcon' 'g6SUUcage' 'x6w6can4a' 'g6mPSTR9_5.0'
 'g6fPSTR12_4.0' 'EDUC_d_7.0' 'c6w6MUSE1_0.0' 'c6w4muse1_7.0'
 'g6fPSTR11_4.0' 'x6mcanp01' 'g6mparmon' 'c6mcanp1' 'c6mcanp05'
 'c6w4use_1.0' 'g6fparsup' 'c6w4muse1_0.0' 'c6w6MUSE1_1.0' 'g6GEN_2.0'
 'MomPrevent' 'g6mcanp01' 'g6mPSTR14_5.0' 'g6mparsup' 'EDUC_d_9.0'
 'c6w6can4d' 'AUD_3Level_0.0' 'g6mcanp05' 'c6

In [39]:
pastYear_use_predictors_regression = final_var
pastYear_use_scores_regression = [train_acc, val_acc, test_acc]

%store pastYear_use_predictors_regression
%store pastYear_use_scores_regression

Stored 'pastYear_use_predictors_regression' (ndarray)
Stored 'pastYear_use_scores_regression' (list)


### Past 3 months use

In [40]:
estimator = LogisticRegressionCV(solver='saga',penalty='l1', Cs=[1], max_iter = 10000, scoring='accuracy',
                   multi_class='multinomial',random_state=0, cv=2, refit=True)
X = X_final
y = y_filtered_df['g6SUUMUSE3']
target = 'g6SUUMUSE3'

LogReg123(estimator, X, y, target)
    

Model # 3 is selected 

The total number of variables is:	 70 

These are the variables:

 ['g6mPSTR12_5.0' 'g6fPSTR16_3.0' 'g6mPSTR14_1.0' 'EDUC_11.0' 'g6fparsup'
 'x6mcanp2' 'g6ETHNIC_1.0' 'c6dep_0.0' 'MomPrevent' 'c6w4canre_-1.0'
 'MomDiscuss' 'g6mPSTR11_4.0' 'g6mparsup' 'g6mcanp01' 'x6w6MUSE1_6.0'
 'DadPrevent' 'c6w5muse3_3.0' 'g6fPSTR14_5.0' 'c6w6use_0.0' 'ethnic_1.0'
 'c6dep_1.0' 'c6w6can4d' 'x6w6MUSE1_5.0' 'x6w6use_1.0' 'g6cage'
 'g6fPSTR12_5.0' 'g6mcanp05' 'DadNegExp' 'c6mcanp2' 'EDUC_6.0'
 'g6mPSTR10_3.0' 'g6mPSTR9_3.0' 'AUD_3Level_1.0' 'x6w6MUSE2_0.0'
 'c6w4canon_-1.0' 'g6fPSTR9_3.0' 'c6cage' 'x6w6can4d' 'g6mcanp1'
 'c6mcanp01' 'g6fparcon' 'g6fPSTR11_1.0' 'c6RELAT_1.0' 'MomNegExp'
 'g6fPSTR10_4.0' 'g6mparcon' 'g6SUUcage' 'x6w6can4a' 'c6w4candx_4.0'
 'x6w6MUSE3_1.0' 'g6fPSTR13_4.0' 'g6mPSTR13_1.0' 'EDUC_d_7.0' 'g6fparmon'
 'c6w6MUSE1_0.0' 'x6use_1.0' 'g6fPSTR9_2.0' 'g6mPSTR15_1.0'
 'g6mPSTR10_5.0' 'c6w6can4a' 'EDUC_m_8.0' 'c6w5muse3_0.0' 'c6w6use_1.0'
 'g6fPSTR13_2.0' 'g6fPSTR

In [41]:
past3mo_use_predictors_regression = final_var
past3mo_use_scores_regression = [train_acc, val_acc, test_acc]

%store past3mo_use_predictors_regression
%store past3mo_use_scores_regression

Stored 'past3mo_use_predictors_regression' (ndarray)
Stored 'past3mo_use_scores_regression' (list)


### General use (yes or no)

In [42]:
estimator = LogisticRegressionCV(solver='saga',penalty='l1', Cs=[1], max_iter = 10000, scoring='accuracy',
                   multi_class='multinomial',random_state=0, cv=3, refit=True)
X = X_final
y = y_filtered_df['MJ_Outcome_YesNo']
target = 'MJ_Outcome_YesNo'

LogRegMJ(estimator, X, y, target)

Model # 3 is selected 

The total number of variables is:	 75 

These are the variables:

 ['g6mPSTR12_5.0' 'DadDiscuss' 'EDUC_d_4.0' 'g6fparsup' 'x6mcanp2'
 'x6RELAT_1.0' 'g6ETHNIC_1.0' 'x6w6MUSE1_7.0' 'MomPrevent' 'g6mcanp01'
 'g6mPSTR11_4.0' 'g6mPSTR15_3.0' 'EDUC_d_9.0' 'c6w5muse3_3.0'
 'g6fPSTR14_3.0' 'c6w6use_0.0' 'ethnic_1.0' 'c6w5candx_4.0' 'g6cage'
 'x6w6MUSE1_5.0' 'EDUC_d_3.0' 'AUD_3Level_0.0' 'g6mcanp05' 'DadNegExp'
 'g6fPSTR12_5.0' 'c6mcanp2' 'EDUC_7.0' 'g6mPSTR10_3.0' 'c6w5muse2_0.0'
 'g6mPSTR11_2.0' 'c6w5muse1_2.0' 'g6ETHNIC_6.0' 'g6fPSTR9_3.0' 'c6cage'
 'x6w6can4d' 'g6fPSTR10_5.0' 'g6mPSTR15_2.0' 'g6mcanp1' 'c6mcanp01'
 'g6fparcon' 'g6fPSTR11_1.0' 'g6mPSTR16_1.0' 'g6fPSTR10_3.0'
 'g6fPSTR12_2.0' 'g6mparcon' 'g6SUUcage' 'x6w6can4a' 'g6mPSTR11_3.0'
 'g6mPSTR12_1.0' 'g6mPSTR9_5.0' 'c6w4muse2_2.0' 'EDUC_m_7.0' 'c6w5use_1.0'
 'EDUC_d_1.0' 'x6w6MUSE1_2.0' 'EDUC_d_7.0' 'c6w6MUSE1_0.0' 'g6fPSTR11_3.0'
 'x6w6MUSE2_6.0' 'c6w4muse1_5.0' 'g6fPSTR12_1.0' 'g6mPSTR16_2.0'
 'c6w5use_0.0'

In [43]:
gen_use_predictors_regression = final_var
gen_use_scores_regression = [train_acc, val_acc, test_acc]

%store gen_use_predictors_regression
%store gen_use_scores_regression

Stored 'gen_use_predictors_regression' (ndarray)
Stored 'gen_use_scores_regression' (list)


## Random Forest

In [44]:
def RandFor(estimator, X, y, target):
    training_score = []
    crossVal_score = []
    testing_score = []
    total_num_var = []
    selected_param = []
    selected_var = []
    
    df = pd.concat([X, y], 1)
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    if target == 'g6SUUMUSE3':
        df = df[df[target]!=5]
        df[target].cat.remove_categories([5], inplace=True)
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), 
                                                        df[target], 
                                                        stratify=df[target], 
                                                        test_size=0.15, random_state=0)
    
    params = {'n_estimators':[2,5,10,50,100,200,500],
              'min_samples_leaf':[.01, .05, .1, .2]}
    
    modSearch = GridSearchCV(estimator, params, cv=2, error_score=np.nan)
    modSearch.fit(X_train, y_train)
    rf = modSearch.best_estimator_.fit(X_train, y_train)

    var = X_train.columns.values[list(np.where(rf.feature_importances_!=0))]
    total_num_var.append(len(var))
    selected_param.append(modSearch.best_params_)
    selected_var.append(np.asarray(var))
    
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    training_score.append(accuracy_score(y_train, y_pred_train))
    testing_score.append(accuracy_score(y_test, y_pred_test))
    crossVal_score.append(modSearch.best_score_)
    
    X_new_train = X_train.loc[:, X_train.columns.isin(var)]
    X_new_test = X_test.loc[:, X_test.columns.isin(var)]
    modSearch.fit(X_new_train, y_train)
    rf = modSearch.best_estimator_.fit(X_new_train, y_train)
    
    var = X_new_train.columns.values[list(np.where(rf.feature_importances_!=0))]
    total_num_var.append(len(var))
    selected_param.append(modSearch.best_params_)
    selected_var.append(np.asarray(var))
    
    y_pred_train = rf.predict(X_new_train)
    y_pred_test = rf.predict(X_new_test)
    training_score.append(accuracy_score(y_train, y_pred_train))
    testing_score.append(accuracy_score(y_test, y_pred_test))
    crossVal_score.append(modSearch.best_score_)
    
    diff = crossVal_score[-1] - crossVal_score[-2]
    
    while diff>0:
        X_new_train = X_train.loc[:, X_train.columns.isin(var)]
        X_new_test = X_test.loc[:, X_test.columns.isin(var)]
        modSearch.fit(X_new_train, y_train)
        rf = modSearch.best_estimator_.fit(X_new_train, y_train)
    
        var = X_new_train.columns.values[list(np.where(rf.feature_importances_!=0))]
        total_num_var.append(len(var))
        selected_param.append(modSearch.best_params_)
        selected_var.append(np.asarray(var))
    
        y_pred_train = rf.predict(X_new_train)
        y_pred_test = rf.predict(X_new_test)
        training_score.append(accuracy_score(y_train, y_pred_train))
        testing_score.append(accuracy_score(y_test, y_pred_test))
        crossVal_score.append(modSearch.best_score_)

        diff = crossVal_score[-1] - crossVal_score[-2]

    
    if total_num_var.count(min(total_num_var))>1:
        which_model = testing_score.index(max(testing_score))
    elif crossVal_score[-1] == max(crossVal_score) or diff > -0.01:
        which_model = len(crossVal_score) - 1
    else:
        which_model = crossVal_score.index(max(crossVal_score))
    
    print("Model #",str(which_model+1),"is selected",
          "\n\nFinal model parameters:\t",
          selected_param[which_model],"\n\nThe total number of variables is:\t",
          len(selected_var[which_model]),"\n\nThese are the variables:\n\n",
          selected_var[which_model],"\n\nNumber of variables in each iteration:\n",
          total_num_var,"\n\nTraining accuracy in each iteration:\n",
          training_score,"\n\nCross-validation accuracy in each iteration:\n",
          crossVal_score,"\n\nTesting accuracy in each iteration:\n",
          testing_score)
    
    global final_var
    final_var = selected_var[which_model]
    
    global train_acc
    train_acc = training_score[which_model]
    
    global test_acc
    test_acc = testing_score[which_model]
    
    global val_acc
    val_acc = crossVal_score[which_model]


### Max use

In [45]:
estimator = RandomForestClassifier(random_state=0, bootstrap=True, oob_score=True,
                                   class_weight={0:1,1:10,2:15,3:15,4:15,5:15,6:15,7:15})
X = x_filled_knn
y = y_filtered_df['g6SUUMUSE1']
target = 'g6SUUMUSE1'

RandFor(estimator, X, y, target)

Model # 2 is selected 

Final model parameters:	 {'min_samples_leaf': 0.01, 'n_estimators': 50} 

The total number of variables is:	 168 

These are the variables:

 ['g6SUUcage' 'g6cage' 'g6fpstrmar' 'g6mpstrmar' 'c6w6can4d' 'c6w6can4a'
 'x6w6can4d' 'x6w6can4a' 'par_fscore' 'parf_score2' 'MomPrevent'
 'MomDiscuss' 'MomNegExp' 'DadPrevent' 'DadDiscuss' 'DadNegExp'
 'g6fparsup' 'g6fparcon' 'g6fparmon' 'g6mparsup' 'g6mparcon' 'g6mparmon'
 'c6cage' 'g2agepar' 'g6mcanp01' 'g6mcanp05' 'g6mcanp1' 'g6mcanp2'
 'x6mcanp01' 'x6mcanp05' 'x6mcanp1' 'x6mcanp2' 'c6mcanp01' 'c6mcanp05'
 'c6mcanp1' 'c6mcanp2' 'c6RELAT_1.0' 'c6RELAT_2.0' 'x6RELAT_1.0'
 'x6RELAT_2.0' 'g6GEN_1.0' 'g6GEN_2.0' 'g6ETHNIC_1.0' 'g6ETHNIC_2.0'
 'g6fPSTR9_1.0' 'g6fPSTR9_3.0' 'g6fPSTR9_5.0' 'g6fPSTR10_1.0'
 'g6fPSTR10_3.0' 'g6fPSTR10_4.0' 'g6fPSTR10_5.0' 'g6fPSTR11_1.0'
 'g6fPSTR11_3.0' 'g6fPSTR11_4.0' 'g6fPSTR11_5.0' 'g6fPSTR12_1.0'
 'g6fPSTR12_2.0' 'g6fPSTR12_4.0' 'g6fPSTR12_5.0' 'g6fPSTR13_1.0'
 'g6fPSTR13_3.0' 'g6fPSTR13_4.0

In [46]:
max_use_predictors_tree = final_var
max_use_scores_tree = [train_acc, val_acc, test_acc]

%store max_use_predictors_tree
%store max_use_scores_tree

Stored 'max_use_predictors_tree' (ndarray)
Stored 'max_use_scores_tree' (list)


### Past year use

In [47]:
estimator = RandomForestClassifier(random_state=0, bootstrap=True, oob_score=True,
                                   class_weight={0:1,1:5,2:20,3:20,4:20,5:20,6:20,7:20})
X = x_filled_knn
y = y_filtered_df['g6SUUMUSE2']
target = 'g6SUUMUSE2'

RandFor(estimator, X, y, target)

Model # 1 is selected 

Final model parameters:	 {'min_samples_leaf': 0.1, 'n_estimators': 100} 

The total number of variables is:	 97 

These are the variables:

 ['g6SUUcage' 'g6cage' 'g6fpstrmar' 'g6mpstrmar' 'c6w6can4d' 'x6w6can4d'
 'par_fscore' 'parf_score2' 'MomPrevent' 'MomDiscuss' 'MomNegExp'
 'DadPrevent' 'DadDiscuss' 'DadNegExp' 'g6fparsup' 'g6fparcon' 'g6fparmon'
 'g6mparsup' 'g6mparcon' 'g6mparmon' 'c6cage' 'g2agepar' 'g6mcanp01'
 'g6mcanp05' 'g6mcanp1' 'g6mcanp2' 'x6mcanp01' 'x6mcanp05' 'x6mcanp1'
 'x6mcanp2' 'c6mcanp01' 'c6mcanp05' 'c6mcanp1' 'c6mcanp2' 'c6RELAT_2.0'
 'x6RELAT_1.0' 'x6RELAT_2.0' 'g6GEN_1.0' 'g6ETHNIC_1.0' 'g6fPSTR9_1.0'
 'g6fPSTR10_5.0' 'g6fPSTR11_1.0' 'g6fPSTR11_5.0' 'g6fPSTR12_1.0'
 'g6fPSTR14_1.0' 'g6fPSTR15_1.0' 'g6fPSTR16_1.0' 'g6mPSTR9_1.0'
 'g6mPSTR10_1.0' 'g6mPSTR10_5.0' 'g6mPSTR11_1.0' 'g6mPSTR11_3.0'
 'g6mPSTR11_5.0' 'g6mPSTR12_1.0' 'g6mPSTR13_1.0' 'g6mPSTR13_5.0'
 'g6mPSTR14_1.0' 'g6mPSTR15_1.0' 'g6mPSTR16_1.0' 'c6w4muse1_0.0'
 'c6w4muse2_0.0'

In [48]:
pastYear_use_predictors_tree = final_var
pastYear_use_scores_tree = [train_acc, val_acc, test_acc]

%store pastYear_use_predictors_tree
%store pastYear_use_scores_tree

Stored 'pastYear_use_predictors_tree' (ndarray)
Stored 'pastYear_use_scores_tree' (list)


### Past 3 months use

In [49]:
estimator = RandomForestClassifier(random_state=0, bootstrap=True, oob_score=True,
                                   class_weight={0:1,1:2,2:5,3:5,4:5,6:5,7:5})
X = x_filled_knn
y = y_filtered_df['g6SUUMUSE3']
target = 'g6SUUMUSE3'

RandFor(estimator, X, y, target)

Model # 2 is selected 

Final model parameters:	 {'min_samples_leaf': 0.01, 'n_estimators': 50} 

The total number of variables is:	 87 

These are the variables:

 ['g6SUUcage' 'g6cage' 'g6fpstrmar' 'g6mpstrmar' 'c6w6can4a' 'x6w6can4d'
 'par_fscore' 'parf_score2' 'MomPrevent' 'MomDiscuss' 'DadPrevent'
 'DadDiscuss' 'DadNegExp' 'g6fparsup' 'g6fparcon' 'g6fparmon' 'g6mparsup'
 'g6mparcon' 'g6mparmon' 'c6cage' 'g2agepar' 'g6mcanp01' 'g6mcanp05'
 'g6mcanp1' 'g6mcanp2' 'x6mcanp01' 'x6mcanp05' 'x6mcanp1' 'x6mcanp2'
 'c6mcanp01' 'c6mcanp05' 'c6mcanp1' 'c6mcanp2' 'x6RELAT_1.0' 'x6RELAT_2.0'
 'g6GEN_1.0' 'g6GEN_2.0' 'g6fPSTR9_1.0' 'g6fPSTR9_4.0' 'g6fPSTR10_4.0'
 'g6fPSTR11_2.0' 'g6fPSTR12_1.0' 'g6fPSTR12_5.0' 'g6fPSTR13_1.0'
 'g6fPSTR14_1.0' 'g6fPSTR14_2.0' 'g6fPSTR14_4.0' 'g6fPSTR16_1.0'
 'g6fPSTR16_3.0' 'g6mPSTR10_1.0' 'g6mPSTR10_3.0' 'g6mPSTR10_5.0'
 'g6mPSTR11_3.0' 'g6mPSTR12_2.0' 'g6mPSTR13_3.0' 'g6mPSTR14_1.0'
 'g6mPSTR14_2.0' 'g6mPSTR16_2.0' 'c6w4muse1_1.0' 'c6w4muse1_2.0'
 'c6w4muse2_0

In [50]:
past3mo_use_predictors_tree = final_var
past3mo_use_scores_tree = [train_acc, val_acc, test_acc]

%store past3mo_use_predictors_tree
%store past3mo_use_scores_tree

Stored 'past3mo_use_predictors_tree' (ndarray)
Stored 'past3mo_use_scores_tree' (list)


### General use (yes or no)

In [51]:
estimator = RandomForestClassifier(random_state=0, bootstrap=True, oob_score=True,
                                   class_weight={0:1,1:10})
X = x_filled_knn
y = y_filtered_df['MJ_Outcome_YesNo']
target = 'MJ_Outcome_YesNo'

RandFor(estimator, X, y, target)

Model # 4 is selected 

Final model parameters:	 {'min_samples_leaf': 0.01, 'n_estimators': 10} 

The total number of variables is:	 60 

These are the variables:

 ['g6SUUcage' 'g6cage' 'g6fpstrmar' 'g6mpstrmar' 'c6w6can4a' 'x6w6can4d'
 'x6w6can4a' 'par_fscore' 'parf_score2' 'MomPrevent' 'MomDiscuss'
 'MomNegExp' 'DadPrevent' 'DadDiscuss' 'DadNegExp' 'g6fparsup' 'g6fparcon'
 'g6fparmon' 'g6mparsup' 'g6mparcon' 'g6mparmon' 'c6cage' 'g2agepar'
 'g6mcanp01' 'g6mcanp1' 'g6mcanp2' 'x6mcanp01' 'x6mcanp05' 'x6mcanp1'
 'x6mcanp2' 'c6mcanp01' 'c6mcanp05' 'c6mcanp1' 'c6mcanp2' 'g6GEN_1.0'
 'g6GEN_2.0' 'g6fPSTR9_5.0' 'g6fPSTR10_5.0' 'g6fPSTR13_1.0'
 'g6fPSTR16_1.0' 'g6mPSTR10_5.0' 'g6mPSTR11_3.0' 'g6mPSTR12_5.0'
 'c6w4canon_-1.0' 'c6w4canre_-1.0' 'c6w6MUSE1_0.0' 'c6w6MUSE1_2.0'
 'x6w6MUSE1_2.0' 'c6w6candep_1.0' 'x6can4_1.0' 'parentdx_1.0'
 'c6w4use_0.0' 'c6use_1.0' 'x6use_1.0' 'parentuse_1.0' 'ParHist_2.0'
 'ethnic_1.0' 'EDUC_m_3.0' 'EDUC_d_3.0' 'w6parentdep_0.0'] 

Number of variables in each i

In [52]:
gen_use_predictors_tree = final_var
gen_use_scores_tree = [train_acc, val_acc, test_acc]

%store gen_use_predictors_tree
%store gen_use_scores_tree

Stored 'gen_use_predictors_tree' (ndarray)
Stored 'gen_use_scores_tree' (list)


## Summary of results

In [53]:
all_scores = [max_use_scores_regression, pastYear_use_scores_regression,
              past3mo_use_scores_regression, gen_use_scores_regression,
              max_use_scores_tree, pastYear_use_scores_tree,
              past3mo_use_scores_tree, gen_use_scores_tree]

score_summary = pd.DataFrame(columns=['Training Accuracy', 'Cross-Validation Accuracy', 'Testing Accuracy'], 
                             data=all_scores)

response = ["Max Use", "Past Year Use", "Past 3 Months Use", "General Use"]
response = response + response
model = ["Logistic Regression", "Random Forest"]
model = np.repeat(model, 4)

In [54]:
score_summary.insert(0, "Model", model, allow_duplicates = True)
score_summary.insert(0, "Variable", response, allow_duplicates = True)
score_summary

,Variable,Model,Training Accuracy,Cross-Validation Accuracy,Testing Accuracy
0,Max Use,Logistic Regression,0.944625,0.726759,0.690909
1,Past Year Use,Logistic Regression,0.941176,0.823610,0.836364
2,Past 3 Months Use,Logistic Regression,0.967320,0.882418,0.870370
3,General Use,Logistic Regression,0.954397,0.863052,0.836364
4,Max Use,Random Forest,0.986971,0.771987,0.727273
5,Past Year Use,Random Forest,0.869281,0.843137,0.836364
6,Past 3 Months Use,Random Forest,0.937908,0.905229,0.907407
7,General Use,Random Forest,0.960912,0.853420,0.836364
